In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import jaccard_score

# Load data
path = "C:/path/to/knn clusters.csv"  # Change this to your actual path
df = pd.read_csv(path)

# Variables to use for clustering
cluster_vars = ['bmi', 'dm_age', 'glycohemoglobin', 'homa2b', 'homa2ir']
df = df.dropna(subset=cluster_vars).reset_index(drop=True)

# Scale the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df[cluster_vars])

# Run original KMeans
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
original_labels = kmeans.fit_predict(data_scaled)

# Store index for tracking
df['orig_cluster'] = original_labels

# For reproducibility
np.random.seed(42)

# Run bootstrapping
n_bootstraps = 100
jaccard_scores = np.zeros((n_bootstraps, n_clusters))

for b in range(n_bootstraps):
    # Sample with replacement
    sample_indices = np.random.choice(df.index, size=len(df), replace=True)
    bootstrap_df = df.loc[sample_indices].copy()
    
    # Re-scale bootstrapped data
    boot_scaled = scaler.transform(bootstrap_df[cluster_vars])
    
    # KMeans on bootstrap sample
    boot_kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
    boot_labels = boot_kmeans.fit_predict(boot_scaled)
    bootstrap_df['boot_cluster'] = boot_labels

    # Find indices present in both datasets (intersection)
    shared_indices = np.intersect1d(df.index, sample_indices)

    # Get labels for shared indices
    orig_labels_shared = df.loc[shared_indices, 'orig_cluster']
    boot_labels_shared = bootstrap_df.loc[bootstrap_df.index.isin(shared_indices), 'boot_cluster'].reset_index(drop=True)

    # Now compare each original cluster with best matching bootstrap cluster
    for orig_c in range(n_clusters):
        orig_mask = (orig_labels_shared == orig_c).astype(int)
        max_jaccard = 0
        for boot_c in range(n_clusters):
            boot_mask = (boot_labels_shared == boot_c).astype(int)
            if len(orig_mask) == len(boot_mask) and np.sum(orig_mask | boot_mask) > 0:
                jaccard = jaccard_score(orig_mask, boot_mask)
                max_jaccard = max(max_jaccard, jaccard)
        jaccard_scores[b, orig_c] = max_jaccard

# Summarize
summary = pd.DataFrame({
    'Cluster': [f'Cluster {i}' for i in range(n_clusters)],
    'Mean_Jaccard': np.mean(jaccard_scores, axis=0),
    'Std_Jaccard': np.std(jaccard_scores, axis=0)
})

print("=== Cluster Stability (Jaccard Index) ===")
print(summary)


/var/folders/4v/nhc7w6ks4wd6tsd_29fvnbz00000gn/T/ipykernel_4506/2310665663.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['cluster'] = df['cluster'].replace(label_map)
/Users/ksanaka/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/ksanaka/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/ksanaka/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/ksanaka/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:237: RuntimeWarning: divide by zero encounte


=== Cluster Stability (Jaccard Index) ===
  Cluster  Mean_Jaccard  Std_Jaccard
Cluster 0           0.0          0.0
Cluster 1           0.0          0.0
Cluster 2           0.0          0.0
Cluster 3           0.0          0.0
